<a target="_blank" href="https://colab.research.google.com/github/sonder-art/fdd_o23/blob/main/codigo/polars/04_ejercicio_polars.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
!pip install polars

In [2]:
import polars as pl
import random
import string


# Generacion de Datos

In [3]:
# Funciones auxiliares para generar datos
def random_string(length=5):
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for i in range(length))

def generate_data(n=1000):
    data = {
        "id_venta": list(range(1, n + 1)),
        "producto": [random_string() for _ in range(n)],
        "cantidad": [random.randint(1, 20) for _ in range(n)],
        "precio_unitario": [random.uniform(10, 100) for _ in range(n)],
        "fecha_venta": [pl.date(random.randint(2010, 2022), random.randint(1, 12), random.randint(1, 28)) for _ in range(n)],
        "cliente_id": [random.randint(1000, 5000) for _ in range(n)]
    }
    return pl.DataFrame(data)

# Generar el DataFrame
df = generate_data()
df.head()

id_venta,producto,cantidad,precio_unitario,fecha_venta,cliente_id
i64,str,i64,f64,object,i64
1,"""najtj""",10,86.898849,"2021-11-18 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",3682
2,"""nqlcn""",4,81.580301,"2017-11-24 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",2234
3,"""qekrj""",13,16.912128,"2012-02-26 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",2730
4,"""mkfih""",11,22.840706,"2013-12-25 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",4887
5,"""sodzc""",8,48.395896,"2014-11-01 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",1469


In [4]:
df = df.lazy()
df

# Limpieza de Datos

Utilizando Polars en modo Lazy, realice las siguientes operaciones de limpieza y manipulación de datos:

    1. `Filtrar` las ventas con cantidades menores a 5.
    2. `Agregar` una nueva columna llamada total_venta, que sea el resultado de multiplicar cantidad por precio_unitario.
    3. `Eliminar` la columna precio_unitario después de crear total_venta.
    4. `Agrupar` los datos por producto y calcular el total vendido y la cantidad total vendida por producto.
    5. `Ordenar` los resultados por el total vendido en orden descendente.

Todo esto lo debes de realizar en una sola linea de codigo

In [27]:

lazy_df = df.lazy()

# Realizar las operaciones de limpieza y manipulación

#resultado = lazy_df.filter(pl.col("cantidad")<5).group_by("producto").agg([
#    pl.col("cantidad").mul(pl.col("precio_unitario")).alias("total_venta")
#])

resultado = (
    lazy_df.filter(pl.col("cantidad")<5).with_columns((pl.col('cantidad') * pl.col('precio_unitario')).alias('total_venta'))
.drop('precio_unitario').group_by('producto').agg([pl.col('total_venta').sum().alias('total_vendido'), 
        pl.col('cantidad').sum().alias('cantidad_total_vendida')]).sort('total_vendido', descending=True)

)
    


# Ejecutar y mostrar los resultados
resultado_df = resultado.collect()
print(resultado_df)


shape: (196, 3)
┌──────────┬───────────────┬────────────────────────┐
│ producto ┆ total_vendido ┆ cantidad_total_vendida │
│ ---      ┆ ---           ┆ ---                    │
│ str      ┆ f64           ┆ i64                    │
╞══════════╪═══════════════╪════════════════════════╡
│ rmfww    ┆ 376.134446    ┆ 4                      │
│ lmews    ┆ 368.95855     ┆ 4                      │
│ wtcyt    ┆ 368.077844    ┆ 4                      │
│ umjto    ┆ 366.740083    ┆ 4                      │
│ wfhzf    ┆ 363.266342    ┆ 4                      │
│ …        ┆ …             ┆ …                      │
│ kzxge    ┆ 16.238829     ┆ 1                      │
│ uovwm    ┆ 15.136625     ┆ 1                      │
│ colyu    ┆ 14.504259     ┆ 1                      │
│ hbgdk    ┆ 14.289877     ┆ 1                      │
│ slrtm    ┆ 11.76023      ┆ 1                      │
└──────────┴───────────────┴────────────────────────┘


# Guarda el resultado en un archivo `parquet`

Guarda el resultado en un archivo parquet que se llame `resultados.parquet`

In [28]:
# Tu codigo aqui
resultado_df.write_parquet("resultados.parquet")

# Lee Parquet

Crea codigo para leer en polars el archivo parquet

In [29]:
# Tu codigo aqui
df_resultados = pl.read_parquet("resultados.parquet")
print(df_resultados)

shape: (196, 3)
┌──────────┬───────────────┬────────────────────────┐
│ producto ┆ total_vendido ┆ cantidad_total_vendida │
│ ---      ┆ ---           ┆ ---                    │
│ str      ┆ f64           ┆ i64                    │
╞══════════╪═══════════════╪════════════════════════╡
│ rmfww    ┆ 376.134446    ┆ 4                      │
│ lmews    ┆ 368.95855     ┆ 4                      │
│ wtcyt    ┆ 368.077844    ┆ 4                      │
│ umjto    ┆ 366.740083    ┆ 4                      │
│ wfhzf    ┆ 363.266342    ┆ 4                      │
│ …        ┆ …             ┆ …                      │
│ kzxge    ┆ 16.238829     ┆ 1                      │
│ uovwm    ┆ 15.136625     ┆ 1                      │
│ colyu    ┆ 14.504259     ┆ 1                      │
│ hbgdk    ┆ 14.289877     ┆ 1                      │
│ slrtm    ┆ 11.76023      ┆ 1                      │
└──────────┴───────────────┴────────────────────────┘
